# Generate Pitch Outcome
 - Given the thrown pitch into the batter, will a ball, strike, or contact be made?
 - This is the first portion of the batting outcome
 - Splitting batting outcome into two parts, (second being if contact is made, what happens?) makes things more clear
 - Will be used as input to the second part of the batter outcome mentioned above

## Potential Difficulties:
 - How to factor in umpire for discrepencies in strikes/balls?
## TODO:
 - Write script to get features (formatted with labels in torch)
 - Build model architecture
 - Hyperparameter optimization
 - Compare with bayesian models
 - Figure out how to solve small sample..

In [ ]:
import torch
from torch import

In [1]:
#get test set
from src.features import build_features as f
vladdy = f.get_pitch_outcome_dataset(665489)

Querying dataset...
/home/reece/sports/mlb_simulator/data/databases/mlb.db


In [7]:
#model = YourModel()  # Initialize your model
#criterion = nn.CrossEntropyLoss()  # Or any other suitable loss function
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs=1
for epoch in range(num_epochs):
    for features, labels in vladdy:
        #optimizer.zero_grad()
        #outputs = model(features)
        #loss = criterion(outputs, labels)
        #loss.backward()
        #optimizer.step()
        print(f'features: {features}\nlabels: {labels}')

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

### Non-Numerical Variables:
 - type (what model is trying to predict)
 - pitch_type (nominal)
 - p_throws (nominal)